# Training sentiment analysis model using Watson NLP

This notebook demonstrates how to train sentiment analysis model on book reviews in the Dutch language using Watson NLP.

The data that is used in this notebook is taken from the `Dutch Book Reviews Dataset`, https://github.com/benjaminvdb/DBRD with license [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/): Public Domain. This dataset is derived from the [Hebban - voor lezers door lezers](https://www.hebban.nl/), containing over 4.3 million book reviews and ratings. The dataset has been downsampled and prepared for fast execution of the notebook. You can download the downsampled and prepared data from [Nederlandstalige Watson NLP GitHub Repo](https://github.com/joost-vos/watson-nlp-handson/blob/main/boek_reviews_xs.csv)

### What you'll learn in this notebook
Watson NLP offers so-called blocks for various NLP tasks. This notebook shows:

- **Sentiment analysis** with the _Sentiment block_ (`BERT Document Sentiment block`). Sentiment analysis classifies the sentiment of the reviews into positive or negative  sentiment. You will use the Sentiment workflow to train a BERT based sentiment analysis model. Then you will save the trained sentiment analysis model and finally evaluated the trained model on the test dataset for book reviews in the Dutch language

## Table of Contents


1.  [Before you start](#beforeYouStart)
1.  [Data Loading](#loadData)
1.  [Data Processing](#processData)
1.  [Model Building](#buildModel)
    1. [Train sentiment analysis model using workflow](#trainWorkflow)
    1. [Save the model/workflow](#saveModel)
1.  [Model Evaluation](#evaluateModel)
1.  [Summary](#summary)

<a id="beforeYouStart"></a>
## 1. Before you start

<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.10 + Watson NLP XS_ environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)


<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

Begin by importing and initializing some helper libs that are used throughout the notebook.

In [ ]:
import os, types
import pandas as pd
# we want to show large text snippets to be able to explore the relevant text
pd.options.display.max_colwidth = 400

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import watson_nlp


<a id="loadData"></a>
## 2. Data Loading (Book reviews)

<div class="alert alert-block alert-info">
<b>Tip:</b> If you want to carry out sentiment analysis on any other dataset, you should first upload the dataset into the project and then update the name of the file in the next cell</div>

We load the book reviews into a DataFrame.
Either choose the extra small (xs) data set containing 1000 book reviews with labels (boek_reviews_xs.csv).
    or
choose the medium (M) data set containing 5000 book reviews with labels (boek_reviews_m.csv).

The XS dataset takes roughly 2-3 minutes per training epoch, whereas the M dataset takes 30 minutes per training epoch.


In [ ]:
# import os, types
# import pandas as pd
buffer = project.get_file("boek_reviews_xs.csv") # or use the boek_reviews_m.csv file  - !note! Training takes much longer with the larger source file

books_df = pd.read_csv(buffer)
books_df.head(20)


<a id="processData"></a>
## 3. Data Processing

Helper function to convert the raw data into processed data which can be fed into the sentiment classification model.

In [ ]:
def convertToList(x):
    return [x]

In [ ]:
def input_data_prep(df):
    df['weight'] = 1
    df.rename(columns={'label': 'labels'}, inplace=True)
    df = df[['text', 'weight', 'labels']]
    df['labels'] = df['labels'].replace({0: 'negative', 1: 'positive'})
    df['labels'] = df['labels'].apply(convertToList)
    display(df.head(10))
    return df

In [ ]:
books_df_processed = input_data_prep(books_df)


Splitting the dataset into test & train. Train dataset will be used to train the sentiment classification model and test dataset will be used to evaluate the model.

In [ ]:
from sklearn.model_selection import train_test_split
train_orig, test_orig = train_test_split(books_df, test_size=0.2)
train, test = train_test_split(books_df_processed, test_size=0.2)

In [ ]:
train_file = './train_data.json'
train.to_json(train_file, orient='records')
    
test_file = './test_data.json'
test.to_json(test_file, orient='records')

test.head(10)

<a id="buildModel"></a>
## 4. Model Building

<a id="trainWorkflow"></a>
## 4.A Train the sentiment analysis model using workflow

In [ ]:
import os
from watson_nlp.workflows.document_sentiment import BERT
from watson_nlp.toolkit import bert_utils

In [ ]:
# Download Pretrained model resource 
pretrained_model_resource = watson_nlp.load(watson_nlp.download("pretrained-model_bert_multi_bert_multi_uncased"))
# Print the number of hidden layers
bert_config = bert_utils.BertConfig.from_json_file(pretrained_model_resource.bert_config_path)
print("num_hidden_layers in base model: {}".format(bert_config.num_hidden_layers))

There are 12 layers in the `pretrained-model_bert_multi_bert_multi_uncased` model. You can modify the number of layers and other hyperparameters as shown in the next cells

In [ ]:
# Load the Sentiment workflow model for Dutch
sentiment_model = watson_nlp.download_and_load('sentiment-aggregated_cnn-workflow_nl_stock')

In [ ]:
# Download Dutch Syntax stock models
print('Downloading Syntax')
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_nl_stock'))

In [ ]:
# syntax_lang_code_map = {"en": syntax_model}
syntax_lang_code_map = {"nl": syntax_model}

<div class="alert alert-block alert-warning">
<b>Warning!</b></div>
<span style="color:red">The next cell is going to take a lot of time in training. You can reduce this argument `train_max_seq_length` to 32 or 64 for quicker training</span>

#### Training hyperparameters & arguments:

- train_file: Name of the json file which will be used for training the model
- test_file: Name of the json file which will be used for evaluating the model
- syntax_lang_code_map: A dictionary with language and syntax model. `{"en": syntax_model}` in this case
- pretrained_model_resource: The name of the pretrained model downloaded from Watson NLP library. `pretrained-model_bert_multi_bert_multi_uncased` in this case.
- label_list: List of labels for sentiment classification. `['negative', 'neutral', 'positive']`
- learning_rate: [Learning rate for the model](https://en.wikipedia.org/wiki/Learning_rate). `2e-5`
- num_train_epochs: Number of times the learning algorithm will work through the entire training dataset. `5`
- do_lower_case: Convert all text to lower case. `True`
- train_max_seq_length: Maximum number of tokens that a training sequence can contain. `128`
- train_batch_size: Training batch size. `32`
- dev_batch_size: Validation batch size. `32`
- predict_batch_size: Test batch size. `128`
- predict_max_seq_length: Maximum number of tokens that a prediction sequence can contain. `64`
- num_layers_to_remove: Number of layers to be removed from the base model. `2`
- combine_approach: "Mean" will average score calculation directly while 'NON_NEUTRAL_MEAN' will ignore neutral sentences for average score calculation. `NON_NEUTRAL_MEAN`
- keep_model_artifacts: Keep model artifacts. `True`

In [ ]:
# Train using the workflow with compression 
# Set number of layers to remove as 2

bert_wkflow = BERT.train(
              train_file,
              test_file,
              syntax_lang_code_map,
              pretrained_model_resource,
              label_list=['negative', 'neutral', 'positive'],
              learning_rate=2e-5,
              num_train_epochs=5,
              do_lower_case=True,
              train_max_seq_length=64,
              train_batch_size=16, # was 32
              dev_batch_size=16, # was 32
              predict_batch_size=64, # was 128
              predict_max_seq_length=128,
              num_layers_to_remove=2,
              combine_approach="NON_NEUTRAL_MEAN",
              keep_model_artifacts=True)

In [ ]:
# BERT config has been updated and saved in pre_train artifacts"
bert_updated_config_path = "bert_pretrain_artifact/bert_config.json"
bert_config_updated  = bert_utils.BertConfig.from_json_file(bert_updated_config_path)
print("num_hidden_layers after compression: {}".format(bert_config_updated.num_hidden_layers))

You can observe here that the number of layers after compression has reduced from 12 to 10 i.e 2 layers were reduced from the base model for faster training.

<a id="saveModel"></a>
## 4.B Save the workflow/model

In [ ]:
# Save the Workflow in a given location
model_path = 'bert_set/wkflow_xs' # model_path = 'bert_set/wkflow'
bert_wkflow.save('bert_set/wkflow_xs') # bert_wkflow.save('bert_set/wkflow')

In [ ]:
# Load the saved workflow for predict
wk_loaded = watson_nlp.load(model_path)

In [ ]:
# Run the workflow with input as raw document and language code
# raw_document = 'I have been looking for this film for ages because it is quite rare to find as it was one of the video nasties. I finally found it on DVD at the end of last year it is a very low budget movie The story is set around amazon jungle tribes that are living in fear of the devil. Laura Crawford is a model who is kidnapped by a gang of thugs while she is working in South America. They take her into the jungle Laura is guarded by some ridiculous native who calls himself "The Devil" she has to go though all unpleasant things until they are happy. Maidens are Chained up. The devil demonstrates eating flesh in a horrible manner. Peter Weston, is the devil hunter, who goes into the jungle to try and rescue her,'
raw_document = "Dit is voor mij het beste boek aller tijden. Kippenvel."
sentiment = wk_loaded.run(raw_document, language_code="nl")
sentiment

Save the model on the Cloud Object Storage (COS) associated with the Watson Studio instance

In [ ]:
project.save_data('bert_wkflow_book_reviews_xs_5_epochs_nl', data=bert_wkflow.as_file_like_object(), overwrite=True)

<a id="evaluateModel"></a>
## 5. Model Evaluation

Helper functions to extract text from the dataset and create a new dataframe with the corresponding sentiment

In [ ]:
def extract_sentiment(review_text):
    # run the syntax model
    syntax_result = syntax_model.run(review_text, parsers=('token', 'lemma', 'part_of_speech'))
    sentiment_result = wk_loaded.run(review_text, sentence_sentiment=True, language_code="nl")
    
    document_sentiment = sentiment_result.to_dict()['label']
    return document_sentiment

# Helper method to create a new dataframe with the corresponding sentiment
def create_sentiment_dataframe(df):
    sentiment = df['text'].apply(lambda text: extract_sentiment(text))
    sentiment_df = pd.DataFrame()
    sentiment_df['Document Sentiment'] = sentiment
    return sentiment_df

In [ ]:
test_small = test_orig

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
sentiment_df = create_sentiment_dataframe(test_small)
books_sentiment_df = test_small[['text', 'labels']].merge(sentiment_df[['Document Sentiment']], how='left', left_index=True, right_index=True)
books_sentiment_df.head()

Using custom function to calculate accuracy. **Note**: model evaluation method is WIP for the workflow based model and will be released soon after which you will not need to write custom code for evaluation.

In [ ]:
import numpy as np
conditions = [
    books_sentiment_df['labels'].eq(0) & books_sentiment_df['Document Sentiment'].eq('SENT_NEGATIVE'),
    books_sentiment_df['labels'].eq(1) & books_sentiment_df['Document Sentiment'].eq('SENT_POSITIVE'),
]

choices = [1,1]

books_sentiment_df['score'] = np.select(conditions, choices, default=0)

In [ ]:
print("ACCURACY SCORE:", books_sentiment_df['score'].sum()/len(books_sentiment_df['score']))

### Confusion Matrix

In [ ]:
# confusion matrix in sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# actual values
books_sentiment_df['labels'] = books_sentiment_df['labels'].replace({0:'SENT_NEGATIVE', 1:'SENT_POSITIVE'})
actual = books_sentiment_df['labels']
# predicted values
predicted = books_sentiment_df['Document Sentiment']

# confusion matrix
matrix = confusion_matrix(actual,predicted, labels=['SENT_POSITIVE','SENT_NEGATIVE'])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=['SENT_POSITIVE','SENT_NEGATIVE']).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(actual,predicted,labels=['SENT_POSITIVE','SENT_NEGATIVE'])
print('Classification report : \n',matrix)

<a id="summary"></a>
## 6. Summary

<span style="color:blue">This notebook demonstrates how to fine-tune/re-train the BERT Sentiment workflow using compression. The model performance has improved over Out-of-the-box (OOTB) pre-trained model where accuracy was 87% and which is now 96% with the fine-tuned model. The model performance generally improves especially if you have nuances in the dataset that your model needs to learn from the dataset.</span>



Please note that this content is made available by IBM Build Lab to foster Embedded AI technology adoption. The content may include systems & methods pending patent with USPTO and protected under US Patent Laws. For redistribution of this content, IBM will use release process. For any questions please log an issue in the hosting [GitHub repository](https://github.com/ibm-build-labs/Watson-NLP). 

Developed by IBM Build Lab 

Copyright - 2022-2023 IBM Corporation